In [56]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
from DataPreparation.DataPreparation import read_data
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from mlpath import mlquest as mlq
mlq.start_quest("StackingEnsemble", table_dest="../../", log_defs=False)

### Read the data

In [57]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', standardize=True)


### Initiate model

In [58]:
clf_rf = RandomForestClassifier(n_estimators=500)
clf_svm = SVC(kernel='linear', C=7)
clf_log = LogisticRegression(multi_class='multinomial', max_iter=1000)

clf = mlq.l(StackingClassifier)(estimators=[('rf', clf_rf), ('svm', clf_svm), ('log', clf_log)], final_estimator=LogisticRegression(multi_class='multinomial', max_iter=1000))

### Train and Evaluate

In [59]:
y_pred = mlq.l(cross_val_predict)(clf, x_data_d, y_data_d, cv=5)
accuracy = np.mean(y_pred == y_data_d)
mlq.log_metrics(accuracy=accuracy.item())
print(classification_report(y_data_d, y_pred))
mlq.end_quest()

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       152
           1       0.95      0.93      0.94       160
           2       0.97      0.97      0.97       324
           3       0.99      0.99      0.99       544

    accuracy                           0.98      1180
   macro avg       0.97      0.97      0.97      1180
weighted avg       0.98      0.98      0.98      1180



### Tracking

In [60]:
mlq.show_logs("StackingEnsemble", table_dest="../../", last_k=6)